# Step 1 Install all python libraries

In [ ]:
!python -m pip install semantic-kernel==0.3.10.dev0
!python -m pip install azure-devops


# Step 2 Import Packages required to prepare a semantic kernel instance first.

In [ ]:
import os
from dotenv import dotenv_values
import semantic_kernel as sk
from semantic_kernel import ContextVariables, Kernel # Context to store variables and Kernel to interact with the kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion # AI services
from semantic_kernel.planning.sequential_planner import SequentialPlanner # Planner

kernel = sk.Kernel() # Create a kernel instance
kernel1 = sk.Kernel() #create another kernel instance for not having semanitc function in the same kernel 

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

# Step 3 Importing semantic skills and function from folder

In [ ]:
# note: using skills from the samples folder
plugins_directory = "./plugins"

    # Import the semantic functions
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureDevOps")
FDesFunction = DevFunctions["FeatureDescription"]  


# Step 4 calling the semantic function with feature title to genrate feature description based on predefined template


In [ ]:
  
resultFD = FDesFunction("Azure Resource Group Configuration Export and Infrastructure as Code (IAC) Generation")

print(resultFD)


# Step 5 Importing native function


In [ ]:
from plugins.AzureDevops.native_function import feature
azdev = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
variables = ContextVariables()


# Step 6 Executing native function by putting natural language queries in title

In [ ]:
variables["title"] = "creating a nice pipelines"
variables["description"] = "test"
result = await kernel.run_async(
                azdev["create"], input_vars=variables
            )
print(result)

# Step 7 Sequential planner to genrate plan to create n number of features to meet requirment

In [ ]:
from plugins.AzureDevops.native_function import feature
planner = SequentialPlanner(kernel)
# Import the native functions
AzDevplugin = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
ask = "create two Azure DevOps features for one with title creating user and one with creating work items with standard feature title and description"
plan = await planner.create_plan_async(goal=ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)


# Step 8 Sequential planner to execute genrated plan using available function

In [ ]:

print("Plan results:")
result = await plan.invoke_async(ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

-Identify unser stories - Identify user stories from feature description

In [ ]:
UDesFunction = DevFunctions["IdentifyUserStory"]    
resultUD = UDesFunction(str(resultFD))

print(resultUD)

Importing and executing **Native plugin** - 
- Create- It creates feature in azure devops features with nested call to semantic function to genrate feature description

Using Basic planner to achive goal by sticking all the fuctions togather.

In [ ]:
import json

subtasks=json.loads(str(plan.generated_plan))
subtasks

Excute plan based on steps from planner function

In [ ]:
# Execute the plan
for index, subtask in enumerate(subtasks, start=1):
    result = await planner.execute_plan_async(plan, kernel)
    print("Plan results:")
    print(result)


**Sequential planner**

In [ ]:
from plugins.AzureDevops.Feature_nested import feature
planner = SequentialPlanner(kernel)
# Import the native functions
math_plugin = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
ask = "create two Azure DevOps features for one with title creating chai and one with creating work items with standard feature title and description"
plan = await planner.create_plan_async(goal=ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)


In [ ]:

print("Plan results:")
result = await plan.invoke_async(ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

**Action Planner**

In [ ]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)
ask = "create identify Azure DevOps features for one with title office work and one with self learning with standard feature title and description"
plan = await planner.create_plan_async(goal=ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

 Execute sequential

In [ ]:
result = await plan.invoke_async()


In [ ]:
result

Importing packages required for preparing and using our our data

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureTextCompletion,
    AzureTextEmbedding,
)


from semantic_kernel.connectors.memory.azure_cognitive_search import (
    AzureCognitiveSearchMemoryStore,
)


Register chat completion end point and Embeded model for preparing memory and quering the data

In [ ]:
kernel = sk.Kernel()

config = dotenv_values(".env")
COLLECTION_NAME = "carml-index-sample"

AZURE_COGNITIVE_SEARCH_ENDPOINT = config["AZURE_COGNITIVE_SEARCH_ENDPOINT"]
AZURE_COGNITIVE_SEARCH_ADMIN_KEY = config["AZURE_COGNITIVE_SEARCH_ADMIN_KEY"]
AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_EM_DEPLOYMENT_NAME = config["AZURE_OPENAI_EM_DEPLOYMENT_NAME"]
AZURE_OPENAI_DEPLOYMENT_NAME = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
vector_size = 1536

kernel.add_text_completion_service(
        "dv",
        AzureTextCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
        ),
    )
kernel.add_text_embedding_generation_service(
       "ada",
        AzureTextEmbedding(
            deployment_name=AZURE_OPENAI_EM_DEPLOYMENT_NAME,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
        ),
    )

Initiating Endpoint for connecting with Azure cognitive search

In [ ]:

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())
connector = AzureCognitiveSearchMemoryStore(
        vector_size, AZURE_COGNITIVE_SEARCH_ENDPOINT, AZURE_COGNITIVE_SEARCH_ADMIN_KEY
    )

    # Register the memory store with the kernel
kernel.register_memory_store(memory_store=connector)


Populating information in cognitive serach using Embeding model

I have some CARML biceps which i am uploading in the cognitive search

In [ ]:
folder_path = (r"C:\Users\vigarudi\Documents\code\Powershell\CARML\Main")  # Replace with the path to your folder

# List all files in the folder
file_list = os.listdir(folder_path)

# Iterate through each file and read its contents
for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            file_contents = file.read()
            # Now, you can work with the file_contents as a string
            await kernel.memory.save_information_async(
                COLLECTION_NAME, id=filename, text=file_contents
                )
            


searching the data in memory 

In [ ]:
#serach in memory
questions = [
        "how to provision VM",
    ]

for question in questions:
    print(f"Question: {question}")
    result = await kernel.memory.search_async(COLLECTION_NAME, question)
    print(f"Answer: {result[0].text}\n")

In [ ]:

await connector.close_async()


embeding the search outcome with semantic function

In [ ]:

sk_prompt = """
    Genrate bicep code to provision {{$user_input}} for example
    Example:
    - {{$fact1}} {{recall $fact1}} 
    - {{$fact1}}
    """.strip()



context = kernel.create_new_context()
context["fact1"] = "How to provision a VM?"
context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = COLLECTION_NAME
context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8
context["user_input"] = "How to provision a VM?"
chat_func = kernel.create_semantic_function(sk_prompt, max_tokens=4000, temperature=0.8)
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

Using Chat completion with our own data set

In [ ]:
import requests
import json
url = "https://ch1-open-ai.openai.azure.com/openai/deployments/ContosoDemos-OpenAI-gpt-35-turbo/extensions/chat/completions?api-version=2023-06-01-preview"
headers =  {"Content-Type":"application/json","api-key": "a33c3a92d6694dedad2bf1e2fb7a9c48" }
data = '''{
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": "https://contoso.search.windows.net",
                "key": "aAEeBL5XE83GVHfJqulOn4BBdRpZJ5J7fizBsAWL6jAzSeAf0yNt",
                "indexName": "carml-index-sample"
            }
        }
    ],
    "messages": [
        {
            "role": "user",
            "content": "Bicep file that contains code to deploy Azure resources, including virtual machines?"
        }
    ]
}'''
response = requests.post(url, data=data, headers=headers)
json_data = json.loads(response.text)
print(str(json_data["choices"][0]['messages'][0]['content']))